In [1]:
import sys
import os

# Get the root directory of the project
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add the root directory to the system path
sys.path.insert(0, project_root)


In [2]:
import pandas as pd 
import os
import json

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer

# import seaborn as sns
# sns.set_theme()

In [3]:
from explainable_dataset import *
from viz_utils import create_highlighted_passage
from IPython.display import display, HTML

def highlight_passage(tokens, binary_list):    
    highlighted_passage = create_highlighted_passage(tokens,
                                                     binary_list,
                                                     None,
                                                     'green',
                                                     'linear')
    display(HTML("\n".join(highlighted_passage)))
    
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')


file_path = '../data/29_random_samples_explained.jsonl'
dataset_me = ExplanationsDataset(file_path, tokenizer, decode_positive_as_list=True)

file_path = '../data/29_random_samples_Meta-Llama-3.1-8B-Instruct.jsonl'
dataset_llama = ExplanationsDataset(file_path, tokenizer, decode_positive_as_list=True)

# Iterate through the data
for i in range(len(dataset_llama)):
    print(dataset_me[i]["query"])
    
    print("Me")
    d = dataset_me[i]
    highlight_passage(d['tokenized_positive_decoded'], d['rationales'])
    
    print("Llma")  
    d = dataset_llama[i]
    highlight_passage(d['tokenized_positive_decoded'], d['rationales'])
    
    print("\n")

what is a system hardware
Me


Llma




meaning of the name lincoln
Me


Llma




do.muslims not believe the holocaust happened
Me


...

Llma




what types of things does facilitated diffusion move
Me


Llma




vitamin d deficiency symptoms in women causes
Me


Llma


In [4]:
import pandas as pd

# Initialize the DataFrame to hold the data
columns = ['query', 'tokenized_positive_decoded_llama', 'rationales_llama',
                    'tokenized_positive_decoded_me', 'rationales_me']

df = pd.DataFrame(columns=columns)

# Iterate through both datasets
for i in range(len(dataset_me)):
    # Get data from both datasets
    data_llama = dataset_llama[i]
    data_me = dataset_me[i]
    
    # Prepare data for the row
    row_data = {
        'query': data_llama['query'],
        'tokenized_positive_decoded_llama': data_llama['tokenized_positive_decoded'],
        'rationales_llama': data_llama['rationales'],
        'tokenized_positive_decoded_me': data_me['tokenized_positive_decoded'],
        'rationales_me': data_me['rationales']
    }
    
    # Append the data to the DataFrame
    df = pd.concat([df, pd.DataFrame([row_data])], ignore_index=True)

# Output the DataFrame
df[:3]


,query,tokenized_positive_decoded_llama,rationales_llama,tokenized_positive_decoded_me,rationales_me
0,what is a system hardware,"[Ab, brev, i, ated, as, H, W, ,, Hardware, is,...","[tensor(0), tensor(0), tensor(0), tensor(0), t...","[Ab, brev, i, ated, as, H, W, ,, Hardware, is,...","[tensor(1), tensor(1), tensor(1), tensor(1), t..."
1,meaning of the name lincoln,"[American, Me, a, ning, :, The, name, Lincoln,...","[tensor(0), tensor(0), tensor(0), tensor(0), t...","[American, Me, a, ning, :, The, name, Lincoln,...","[tensor(0), tensor(0), tensor(0), tensor(0), t..."
2,do.muslims not believe the holocaust happened,"[For, the, majority, of, Muslim, s, in, the, w...","[tensor(1), tensor(1), tensor(1), tensor(1), t...","[For, the, majority, of, Muslim, s, in, the, w...","[tensor(1), tensor(1), tensor(1), tensor(1), t..."


In [5]:
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd

def fill_with_zeroes(xs, ys):      
    start_xs_len = len(xs)
    start_ys_len = len(ys)
    if len(ys) == len(xs):
        return xs, ys
    
    if len(ys) < len(xs):
        xs, ys = ys, xs
    
    xs.append(0)
    if len(xs) < len(ys):
        xs.insert(0, 0) # (to 0th place, insert 0)
        
    if len(xs) < len(ys):
        xs.insert(0, 0) # (to 0th place, insert 0)
    
    if len(xs) != len(ys):
        print(f"They have had different lenghts by more than 3: {start_xs_len}, {start_ys_len}")
    
    return xs, ys
    
def compute_stats(rationales_label):
    # Initialize variables to store the cumulative metrics
    precisions = []
    recalls = []
    f1_scores = []

    # Iterate through each row in the DataFrame
    for index, row in df.iterrows():
        # Get the rationales from both datasets
        rationales_llama = row[rationales_label]
        rationales_me = row['rationales_me']

        y_true = rationales_me.tolist()
        y_pred = rationales_llama.tolist() 

        y_true, y_pred = fill_with_zeroes(y_true, y_pred)

        # Calculate precision, recall, and F1 score for this pair of rationales
        try:
            precision = precision_score(y_true, y_pred)
            recall = recall_score(y_true, y_pred)
            f1 = f1_score(y_true, y_pred)
        except ValueError as e:
            print(f"{index}: {e}")
            print()
            continue
        # Append the scores to the lists
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)

    # Calculate the average of each metric
    avg_precision = sum(precisions) / len(precisions)
    avg_recall = sum(recalls) / len(recalls)
    avg_f1 = sum(f1_scores) / len(f1_scores)

    # Print the averages
    print(f"\n\tStats for {rationales_label}")
    print(f"Average Precision: {avg_precision:.4f}")
    print(f"Average Recall: {avg_recall:.4f}")
    print(f"Average F1 Score: {avg_f1:.4f}")

compute_stats('rationales_llama')
    


	Stats for rationales_llama
Average Precision: 0.5734
Average Recall: 0.6343
Average F1 Score: 0.5635


In [6]:
print(df.iloc[24]["tokenized_positive_decoded_llama"])
print(df.iloc[23]["tokenized_positive_decoded_me"])

['But', 'for', 'businesses', 'you', 'will', 'need', 'to', 'keep', 'them', 'a', 'bit', 'longer', ':', 'for', 'sole', 'trader', 's', 'or', 'limited', 'li', 'ability', 'companies', ',', 'you', 'should', 'keep', 'the', 'records', 'of', 'account', 's', ',', 'ex', 'pendi', 'ture', 'and', 'rece', 'ip', 'ts', 'for', 'at', 'least', '6', 'years', '', ',', 'not', 'including', 'the', 'year', 'you', 'are', 'in', '.', 'For', 'VAT', 'records', 'also', 'you', 'will', 'need', 'to', 'keep', 'your', 'records', 'for', 'a', 'minimum', 'of', '6', 'years', '.', 'he', 'In', 'land', 'Re', 'venue', '(', 'now', '', 'HM', 'RC', ')', 'gives', 'guidelines', 'on', 'this', ',', 'and', 'says', 'that', 'for', 'self', 'assessment', 'return', 's', 'you', 'need', 'to', 'keep', 'your', 'records', 'for', '5', 'full', 'years', 'after', 'the', 'latest', 'date', 'for', 'fi', 'ling', 'the', 'return', '.']
['Steven', 'Bli', 's', 's', '.', 'Stu', 'cco', 'on', 'building', 'exterior', 's', '', ',', 'in', 'its', 'traditional', 'form